In [1]:
# ================================================================
# VGG-16 on CIFAR-10  ▸  HRank-style filter analysis (no training)
# ================================================================
import torch, torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

# ─────────────────── Device ───────────────────
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ─────────────────── Model ────────────────────
# ImageNet-pre-trained weights ⇒ good, stable feature maps.
vgg16 = models.vgg16_bn(weights=models.VGG16_BN_Weights.IMAGENET1K_V1)
vgg16.avgpool = nn.AdaptiveAvgPool2d((1, 1))      # keep features, shrink heads
vgg16.classifier = nn.Sequential(                 # CIFAR-10 head (unused here)
    nn.Flatten(), nn.Linear(512, 10))
model = vgg16.to(device).eval()

# ───────────────── Dataset / Loader ────────────
transform = transforms.Compose([
    transforms.Resize(224),                       # keep ImageNet spatial size
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010)),
])
dataset = datasets.CIFAR10(root="./data",
                           train=False, download=True,
                           transform=transform)
loader  = DataLoader(dataset, batch_size=1, shuffle=True)

# ───────────────── Conv-layer bookkeeping ──────
conv_modules  = [layer for layer in model.features if isinstance(layer, nn.Conv2d)]
conv_ranks_sum = [[0.0] * conv.out_channels for conv in conv_modules]

# ───────────────── Rank accumulation ───────────
num_images, processed = 100, 0                    # ≈1–2 min on GPU
with torch.no_grad():
    for imgs, _ in loader:
        imgs = imgs.to(device)
        x, conv_feats = imgs, []                  # capture every Conv2d output
        for layer in model.features:
            x = layer(x)
            if isinstance(layer, nn.Conv2d):
                conv_feats.append(x)

        # per-filter matrix-rank
        for l_idx, fmap in enumerate(conv_feats):
            fmap = fmap.cpu().squeeze(0)          # (C,H,W)
            for f in range(fmap.shape[0]):
                rank = torch.linalg.matrix_rank(fmap[f]).item()
                conv_ranks_sum[l_idx][f] += rank

        processed += 1
        if processed >= num_images:
            break

# ───────────────── Console summary ─────────────
for l_idx, conv in enumerate(conv_modules, start=1):
    in_ch, out_ch = conv.in_channels, conv.out_channels
    avg_ranks = [r / processed for r in conv_ranks_sum[l_idx-1]]
    t = torch.tensor(avg_ranks)
    hi, lo = t.max().item(), t.min().item()
    hi_ids = (t == hi).nonzero(as_tuple=True)[0].tolist()
    lo_ids = (t == lo).nonzero(as_tuple=True)[0].tolist()

    print(f"Layer {l_idx} (Conv2d {in_ch}->{out_ch}):")
    for f, a in enumerate(avg_ranks):
        print(f"  Filter {f:3d}: average rank {a:5.3f}")
    print(f"  --> Highest avg rank {hi:5.3f} at filter(s) {hi_ids}")
    print(f"  --> Lowest  avg rank {lo:5.3f} at filter(s) {lo_ids}\n")


Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth
100%|██████████| 528M/528M [00:02<00:00, 191MB/s]
100%|██████████| 170M/170M [00:13<00:00, 12.5MB/s]


Layer 1 (Conv2d 3->64):
  Filter   0: average rank 156.140
  Filter   1: average rank 166.230
  Filter   2: average rank 147.770
  Filter   3: average rank 148.940
  Filter   4: average rank 161.540
  Filter   5: average rank 138.260
  Filter   6: average rank 131.070
  Filter   7: average rank 151.370
  Filter   8: average rank 131.640
  Filter   9: average rank 152.350
  Filter  10: average rank 141.310
  Filter  11: average rank 134.200
  Filter  12: average rank 145.630
  Filter  13: average rank 182.610
  Filter  14: average rank 133.210
  Filter  15: average rank 139.430
  Filter  16: average rank 170.230
  Filter  17: average rank 152.590
  Filter  18: average rank 190.100
  Filter  19: average rank 160.420
  Filter  20: average rank 153.860
  Filter  21: average rank 139.570
  Filter  22: average rank 140.510
  Filter  23: average rank 137.290
  Filter  24: average rank 136.040
  Filter  25: average rank 147.910
  Filter  26: average rank 87.950
  Filter  27: average rank 145.6

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns, json, os, shutil
from google.colab import files      # Colab-only helper

# ───────────────────── Output dir ─────────────────────
os.makedirs("hrank_output", exist_ok=True)
results_json = {}

# ───────────────────── Histograms & JSON ──────────────
for l_idx, conv in enumerate(conv_modules, start=1):
    in_ch, out_ch = conv.in_channels, conv.out_channels
    avg_ranks = [r / processed for r in conv_ranks_sum[l_idx-1]]

    layer_tag = f"Layer_{l_idx}_Conv2d_{in_ch}to{out_ch}"
    results_json[layer_tag] = {
        "in_channels":  in_ch,
        "out_channels": out_ch,
        "avg_ranks":    avg_ranks,
    }

    plt.figure(figsize=(8, 4))
    sns.histplot(avg_ranks, bins=range(int(max(avg_ranks))+2), kde=False)
    plt.title(f"{layer_tag} – average filter-rank distribution")
    plt.xlabel("Rank");  plt.ylabel("Frequency")
    plt.tight_layout()
    plt.savefig(f"hrank_output/{layer_tag}_hist.png")
    plt.close()

# ───────────────────── JSON / ZIP / Download ──────────
with open("hrank_output/filter_ranks.json", "w") as f:
    json.dump(results_json, f, indent=4)

shutil.make_archive("hrank_output", "zip", "hrank_output")
files.download("hrank_output.zip")
print("✅ All plots and JSON saved to 'hrank_output/' and zipped for download.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ All plots and JSON saved to 'hrank_output/' and zipped for download.
